In [21]:
import os
import sys

# 현재 경로에서 상위 디렉토리(프로젝트 루트)로 이동
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# 기존 경로에서 중복된 항목들을 제거하고 프로젝트 루트를 맨 앞에 추가
sys.path = [project_root] + list(dict.fromkeys(sys.path))

import folium
import math
import numpy as np
from folium.map import Marker
from folium.features import DivIcon
from src.db.maria_client import fetch_orders
from src.model.distance import distance
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from geopy.distance import geodesic

In [22]:
df = fetch_orders()
points = [
    {
        "id": int(row["id"]),
        "latitude": row["latitude"],
        "longitude": row["longitude"],
        "address1": row["address1"],
        "address2": row["address2"],
    }
    for _, row in df.iterrows()
]

depots = [
    {"id": "depot_화성", "latitude": 37.157541, "longitude": 126.834845, "address1": "화성 허브센터", "address2": "", "demand": 0},
    {"id": "depot_이천", "latitude": 37.201687, "longitude": 127.459751, "address1": "이천 마장센터", "address2": "", "demand": 0},
    {"id": "depot_수원", "latitude": 37.263573, "longitude": 127.028601, "address1": "수원센터", "address2": "", "demand": 0},
    {"id": "depot_천안", "latitude": 36.815076, "longitude": 127.113818, "address1": "천안센터", "address2": "", "demand": 0},
    {"id": "depot_일산", "latitude": 37.705338, "longitude": 126.769166, "address1": "일산센터", "address2": "", "demand": 0},
    {"id": "depot_하남", "latitude": 37.535095, "longitude": 127.206303, "address1": "하남센터", "address2": "", "demand": 0},
    {"id": "depot_광주", "latitude": 37.413294, "longitude": 127.255902, "address1": "광주센터", "address2": "", "demand": 0},
    {"id": "depot_기흥", "latitude": 37.274234, "longitude": 127.115570, "address1": "기흥센터", "address2": "", "demand": 0}
]

In [23]:
# 필요한 라이브러리 임포트
import numpy as np  # 수치 계산을 위한 라이브러리
from sklearn.cluster import KMeans  # 군집화를 위한 라이브러리
from geopy.distance import geodesic  # 위경도 기반 거리 계산
import folium  # 지도 시각화
from folium.map import Marker
from folium.features import DivIcon
import math  # 수학 연산
from ortools.constraint_solver import pywrapcp, routing_enums_pb2  # Google OR-Tools (경로 최적화)
from functools import lru_cache  # 캐싱 데코레이터
import concurrent.futures  # 병렬 처리
from datetime import datetime
import os
import sys

# 프로젝트 루트 디렉토리 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path = [project_root] + list(dict.fromkeys(sys.path))

# 데이터베이스 관련 모듈 임포트
from src.db.maria_client import fetch_orders
from src.model.distance import distance

# 거리 계산 함수에 캐싱 적용 (동일한 입력에 대해 재계산 방지)
@lru_cache(maxsize=10000)
def calculate_distance(point1, point2):
    """
    두 지점 간의 거리를 계산하는 함수
    Args:
        point1: (위도, 경도) 튜플
        point2: (위도, 경도) 튜플
    Returns:
        float: 두 지점 간의 거리(km)
    """
    return geodesic(point1, point2).km

def balance_vehicle_loads(points, depot, num_vehicles, max_working_minutes=420):
    """
    시간 밸런싱을 고려하여 차량별로 배송지점을 할당하는 함수
    
    Args:
        points: 배송지점 리스트
        depot: 물류센터 정보
        num_vehicles: 사용 가능한 차량 수
        max_working_minutes: 최대 근무 시간(분)
    
    Returns:
        list: 차량별로 할당된 배송지점 리스트
    """
    # 각 배송지점까지의 예상 소요 시간 계산
    point_times = []
    for point in points:
        # depot부터 배송지점까지의 거리 계산
        dist = calculate_distance(
            (depot['latitude'], depot['longitude']),
            (point['latitude'], point['longitude'])
        )
        # 왕복 시간 + 서비스 시간(5분) 계산
        # 속도는 30km/h로 가정
        time = (dist * 2 * 60 / 30) + 5
        point_times.append((point, time))
    
    # 소요 시간이 긴 순서대로 정렬
    point_times.sort(key=lambda x: x[1], reverse=True)
    
    # 차량별 배송지점 및 시간 초기화
    vehicle_loads = [[] for _ in range(num_vehicles)]  # 차량별 배송지점
    vehicle_times = [0] * num_vehicles  # 차량별 누적 시간
    
    # 가장 긴 시간이 필요한 배송지점부터 차례로 할당
    for point, time in point_times:
        # 현재 누적 시간이 가장 적은 차량 선택
        min_time_vehicle = min(range(num_vehicles), key=lambda i: vehicle_times[i])
        
        # 선택된 차량에 배송지점 할당
        vehicle_loads[min_time_vehicle].append(point)
        vehicle_times[min_time_vehicle] += time
    
    return vehicle_loads

def optimize_vehicle_route(depot, points, vehicle_capacity, max_working_minutes=420, avg_speed_kmh=30):
    """
    단일 차량의 경로를 최적화하는 함수
    
    Args:
        depot: 물류센터 정보
        points: 배송지점 리스트
        vehicle_capacity: 차량 용량
        max_working_minutes: 최대 근무 시간(분)
        avg_speed_kmh: 평균 이동 속도(km/h)
    
    Returns:
        dict: 최적화된 경로 정보
    """
    if not points:
        return None
    
    # depot과 배송지점을 합친 전체 경로 포인트
    all_points = [depot] + points
    size = len(all_points)
    
    # 거리 행렬 계산
    coordinates = [(p['latitude'], p['longitude']) for p in all_points]
    dist_matrix = np.zeros((size, size))
    for i in range(size):
        for j in range(i + 1, size):
            dist = calculate_distance(coordinates[i], coordinates[j])
            dist_matrix[i][j] = dist * 1000  # km -> m 변환
            dist_matrix[j][i] = dist * 1000
    
    # OR-Tools 라우팅 모델 설정
    manager = pywrapcp.RoutingIndexManager(size, 1, 0)  # size, 차량수, 시작점
    routing = pywrapcp.RoutingModel(manager)
    
    # 거리 콜백 함수 정의
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(dist_matrix[from_node][to_node])
    
    # 거리 콜백 등록
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # 시간 제약 설정
    def time_callback(from_index, to_index):
        dist = distance_callback(from_index, to_index)
        travel_time = dist / 1000 * (60 / avg_speed_kmh)  # 이동 시간
        service_time = 5 if manager.IndexToNode(to_index) > 0 else 0  # 서비스 시간
        return int(travel_time + service_time)
    
    # 시간 콜백 등록 및 제약 조건 설정
    time_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.AddDimension(
        time_callback_index,
        10,  # 허용 대기 시간
        max_working_minutes,  # 최대 근무 시간
        True,  # 시작 시간 고정
        'Time'
    )
    
    # 시간 비용 설정
    time_dimension = routing.GetDimensionOrDie('Time')
    time_dimension.SetGlobalSpanCostCoefficient(100)
    
    # 경로 탐색 파라미터 설정
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.seconds = 2  # 탐색 시간 제한
    
    # 경로 최적화 실행
    solution = routing.SolveWithParameters(search_parameters)
    
    if not solution:
        return None
    
    # 최적화된 경로 추출
    route = []
    route_distance = 0
    route_time = 0
    index = routing.Start(0)
    
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        route.append(all_points[node_index])
        
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        
        if not routing.IsEnd(index):
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
            route_time += time_callback(previous_index, index)
    
    return {
        'route': route,
        'distance': route_distance / 1000.0,  # m -> km 변환
        'time': route_time
    }

def create_clusters(points, depot, num_vehicles):
    """
    지역 기반으로 배송지점을 클러스터링하는 함수
    - 방향성과 거리를 고려하여 구역을 분할
    """
    if not points:
        return []
    
    # 각 포인트의 각도와 거리 계산
    point_angles = []
    for point in points:
        # depot 기준 각도 계산
        angle = math.atan2(
            point['latitude'] - depot['latitude'],
            point['longitude'] - depot['longitude']
        )
        # depot부터의 거리 계산
        dist = calculate_distance(
            (depot['latitude'], depot['longitude']),
            (point['latitude'], point['longitude'])
        )
        point_angles.append((point, angle, dist))
    
    # 각도 기준 정렬 (-π ~ π)
    point_angles.sort(key=lambda x: x[1])
    
    # 각도와 거리를 고려한 균등 분할
    clusters = [[] for _ in range(num_vehicles)]
    total_workload = sum(pa[2] for pa in point_angles)  # 총 거리
    target_workload = total_workload / num_vehicles     # 차량당 목표 거리
    
    current_cluster = 0
    current_workload = 0
    
    for point, angle, dist in point_angles:
        clusters[current_cluster].append(point)
        current_workload += dist
        
        # 현재 클러스터의 작업량이 목표치를 초과하면 다음 클러스터로
        if current_workload >= target_workload and current_cluster < num_vehicles - 1:
            current_cluster += 1
            current_workload = 0
    
    return clusters

def optimize_vehicle_route(depot, points, vehicle_capacity, max_working_minutes=420, avg_speed_kmh=30):
    """
    단일 차량의 경로를 최적화하는 함수
    - 경로 겹침 패널티 추가
    - 시간 효율성 개선
    """
    if not points:
        return None
    
    all_points = [depot] + points
    size = len(all_points)
    
    # 거리 행렬 계산
    coordinates = [(p['latitude'], p['longitude']) for p in all_points]
    dist_matrix = np.zeros((size, size))
    
    for i in range(size):
        for j in range(i + 1, size):
            dist = calculate_distance(coordinates[i], coordinates[j])
            dist_matrix[i][j] = dist * 1000
            dist_matrix[j][i] = dist * 1000
    
    manager = pywrapcp.RoutingIndexManager(size, 1, 0)
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(dist_matrix[from_node][to_node])
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # 시간 제약 설정 개선
    def time_callback(from_index, to_index):
        dist = distance_callback(from_index, to_index)
        travel_time = dist / 1000 * (60 / avg_speed_kmh)
        
        # 배송지점 타입에 따른 서비스 시간 차등 적용
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        
        # depot이 아닌 경우에만 서비스 시간 추가
        service_time = 5 if to_node > 0 else 0
        
        return int(travel_time + service_time)
    
    time_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.AddDimension(
        time_callback_index,
        10,  # 허용 대기 시간
        max_working_minutes,
        True,
        'Time'
    )
    
    # 시간 제약 가중치 설정
    time_dimension = routing.GetDimensionOrDie('Time')
    time_dimension.SetGlobalSpanCostCoefficient(100)
    
    # 경로 최적화 파라미터 설정
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.seconds = 3
    
    solution = routing.SolveWithParameters(search_parameters)
    
    if not solution:
        return None
    
    # 최적화된 경로 추출
    route = []
    route_distance = 0
    route_time = 0
    index = routing.Start(0)
    
    while not routing.IsEnd(index):
        node_index = manager.IndexToNode(index)
        route.append(all_points[node_index])
        
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        
        if not routing.IsEnd(index):
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
            route_time += time_callback(previous_index, index)
    
    return {
        'route': route,
        'distance': route_distance / 1000.0,
        'time': route_time
    }

def optimize_routes_for_depot(depot, points, vehicle_capacity=35):
    """
    특정 물류센터의 모든 배송 경로를 최적화하는 함수
    - 클러스터링 방식 개선
    - 시간 밸런싱 강화
    """
    if not points:
        return [], {}
    
    num_vehicles = math.ceil(len(points) / vehicle_capacity)
    
    # 지역 기반 클러스터링 적용
    clusters = create_clusters(points, depot, num_vehicles)
    
    routes = []
    total_distance = 0
    total_time = 0
    vehicle_times = []
    
    for vehicle_id, cluster in enumerate(clusters):
        if not cluster:
            continue
        
        route_info = optimize_vehicle_route(
            depot, 
            cluster, 
            vehicle_capacity,
            max_working_minutes=420,
            avg_speed_kmh=30
        )
        
        if route_info:
            route_info['vehicle_id'] = vehicle_id
            routes.append(route_info)
            total_distance += route_info['distance']
            total_time += route_info['time']
            vehicle_times.append(route_info['time'])
    
    # 시간 통계 계산
    time_std = np.std(vehicle_times) if vehicle_times else 0
    time_mean = np.mean(vehicle_times) if vehicle_times else 0
    
    stats = {
        'depot_id': depot['id'],
        'total_points': len(points),
        'total_vehicles': len(routes),
        'total_distance': total_distance,
        'total_time': total_time,
        'avg_distance_per_vehicle': total_distance / len(routes) if routes else 0,
        'avg_time_per_vehicle': total_time / len(routes) if routes else 0,
        'time_efficiency': total_time / (len(routes) * 420) if routes else 0,
        'time_std': time_std,
        'time_cv': (time_std / time_mean * 100) if time_mean > 0 else 0
    }
    
    return routes, stats

    """
    특정 물류센터의 모든 배송 경로를 최적화하는 함수
    
    Args:
        depot: 물류센터 정보
        points: 배송지점 리스트
        vehicle_capacity: 차량당 최대 배송지점 수
    
    Returns:
        tuple: (최적화된 경로 리스트, 통계 정보)
    """
    if not points:
        return [], {}
    
    # 필요한 차량 수 계산
    num_vehicles = math.ceil(len(points) / vehicle_capacity)
    
    # 시간 밸런싱을 고려하여 배송지점 분배
    balanced_points = balance_vehicle_loads(points, depot, num_vehicles)
    
    # 결과 저장용 변수 초기화
    routes = []
    total_distance = 0
    total_time = 0
    vehicle_times = []
    
    # 각 차량별 경로 최적화
    for vehicle_id, cluster in enumerate(balanced_points):
        if not cluster:
            continue
        
        # 개별 차량 경로 최적화
        route_info = optimize_vehicle_route(
            depot, 
            cluster, 
            vehicle_capacity,
            max_working_minutes=420,  # 7시간
            avg_speed_kmh=30
        )
        
        if route_info:
            route_info['vehicle_id'] = vehicle_id
            routes.append(route_info)
            total_distance += route_info['distance']
            total_time += route_info['time']
            vehicle_times.append(route_info['time'])
    
    # 시간 관련 통계 계산
    time_std = np.std(vehicle_times) if vehicle_times else 0
    time_mean = np.mean(vehicle_times) if vehicle_times else 0
    
    # 통계 정보 생성
    stats = {
        'depot_id': depot['id'],
        'total_points': len(points),
        'total_vehicles': len(routes),
        'total_distance': total_distance,
        'total_time': total_time,
        'avg_distance_per_vehicle': total_distance / len(routes) if routes else 0,
        'avg_time_per_vehicle': total_time / len(routes) if routes else 0,
        'time_efficiency': total_time / (len(routes) * 420) if routes else 0,
        'time_std': time_std,  # 소요시간 표준편차
        'time_cv': (time_std / time_mean * 100) if time_mean > 0 else 0  # 변동계수
    }
    
    return routes, stats

def visualize_routes(routes, depots):
    """
    최적화된 경로를 지도에 시각화하는 함수
    
    Args:
        routes: 최적화된 경로 리스트
        depots: 물류센터 리스트
    
    Returns:
        folium.Map: 시각화된 지도 객체
    """
    if not routes or not depots:
        return None
    
    # 지도 중심점 계산
    center_lat = np.mean([d['latitude'] for d in depots])
    center_lon = np.mean([d['longitude'] for d in depots])
    
    # 기본 지도 생성
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)
    
    # 경로별 색상 설정
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 
              'lightblue', 'darkblue', 'darkgreen', 'cadetblue']
    
    # 물류센터 마커 추가
    for depot in depots:
        folium.Marker(
            location=[depot['latitude'], depot['longitude']],
            popup=f"Depot: {depot['id']}",
            icon=folium.Icon(color='black', icon='info-sign')
        ).add_to(m)
    
    # 각 경로 시각화
    for route_info in routes:
        color = colors[route_info['vehicle_id'] % len(colors)]
        route = route_info['route']
        
        # 경로 라인 그리기
        locations = [[p['latitude'], p['longitude']] for p in route]
        folium.PolyLine(
            locations=locations,
            color=color,
            weight=2,
            opacity=0.8,
            popup=f"Distance: {route_info['distance']:.1f}km, Time: {route_info['time']:.0f}min"
        ).add_to(m)
        
        # 배송지점 마커 추가
        for i, point in enumerate(route[1:], 1):
            folium.CircleMarker(
                location=[point['latitude'], point['longitude']],
                radius=6,
                color=color,
                fill=True,
                popup=f"Stop {i}: {point.get('id', 'N/A')}"
            ).add_to(m)
    
    return m

def main(points, depots):
    """
    메인 실행 함수 - 상세한 결과 출력
    """
    # 수원 물류센터 선택
    suwon_depot = next(d for d in depots if d['id'] == 'depot_수원')
    
    print("\n=== 초기 데이터 정보 ===")
    print(f"전체 배송지점 수: {len(points):,}개")
    print(f"물류센터 수: {len(depots)}개")
    
    # 수원 지역 배송지점 필터링
    suwon_points = []
    for point in points:
        dist = calculate_distance(
            (point['latitude'], point['longitude']),
            (suwon_depot['latitude'], suwon_depot['longitude'])
        )
        if dist <= 15:  # 15km 반경
            suwon_points.append(point)
    
    print(f"\n=== 수원 센터 기본 정보 ===")
    print(f"센터 위치: ({suwon_depot['latitude']:.6f}, {suwon_depot['longitude']:.6f})")
    print(f"배송권역 반경: 15km")
    print(f"배송지점 수: {len(suwon_points):,}개")
    
    # 경로 최적화 실행
    routes, stats = optimize_routes_for_depot(suwon_depot, suwon_points)
    
    print("\n=== 최적화 결과 상세 분석 ===")
    print("\n1. 전체 운영 통계")
    print(f"- 총 배송지점: {stats['total_points']:,}개")
    print(f"- 투입 차량 수: {stats['total_vehicles']}대")
    print(f"- 총 이동거리: {stats['total_distance']:.1f}km")
    print(f"- 총 소요시간: {stats['total_time']:.0f}분 ({stats['total_time']/60:.1f}시간)")
    print(f"- 시간 효율성: {stats['time_efficiency']:.1%}")
    
    print("\n2. 차량당 평균 통계")
    print(f"- 평균 배송지점: {stats['total_points']/stats['total_vehicles']:.1f}개")
    print(f"- 평균 이동거리: {stats['avg_distance_per_vehicle']:.1f}km")
    print(f"- 평균 소요시간: {stats['avg_time_per_vehicle']:.0f}분 ({stats['avg_time_per_vehicle']/60:.1f}시간)")
    print(f"- 소요시간 표준편차: {stats['time_std']:.1f}분")
    print(f"- 소요시간 변동계수: {stats['time_cv']:.1f}%")
    
    print("\n3. 개별 차량 상세 정보")
    times = []
    distances = []
    stops = []
    
    for i, route in enumerate(routes, 1):
        times.append(route['time'])
        distances.append(route['distance'])
        stops.append(len(route['route'])-1)  # depot 제외
        
        print(f"\n[차량 {i}번]")
        print(f"- 배송지점 수: {len(route['route'])-1}개")
        print(f"- 이동거리: {route['distance']:.1f}km")
        print(f"- 소요시간: {route['time']:.0f}분 ({route['time']/60:.1f}시간)")
        
        # 경로 상세 정보
        print("- 상세 경로:")
        for j, point in enumerate(route['route']):
            if j == 0:
                print(f"  출발: {point['address1']} ({point['latitude']:.6f}, {point['longitude']:.6f})")
            else:
                print(f"  배송{j}: {point.get('address1', 'N/A')} ({point['latitude']:.6f}, {point['longitude']:.6f})")
    
    print("\n4. 시간 분포 분석")
    print(f"- 최소 소요시간: {min(times):.0f}분 ({min(times)/60:.1f}시간)")
    print(f"- 최대 소요시간: {max(times):.0f}분 ({max(times)/60:.1f}시간)")
    print(f"- 시간 범위: {max(times) - min(times):.0f}분")
    print(f"- 평균 소요시간: {np.mean(times):.0f}분 ({np.mean(times)/60:.1f}시간)")
    print(f"- 중앙값 소요시간: {np.median(times):.0f}분")
    
    print("\n5. 거리 분포 분석")
    print(f"- 최소 이동거리: {min(distances):.1f}km")
    print(f"- 최대 이동거리: {max(distances):.1f}km")
    print(f"- 거리 범위: {max(distances) - min(distances):.1f}km")
    print(f"- 평균 이동거리: {np.mean(distances):.1f}km")
    print(f"- 중앙값 이동거리: {np.median(distances):.1f}km")
    
    print("\n6. 배송지점 분포 분석")
    print(f"- 최소 배송지점: {min(stops)}개")
    print(f"- 최대 배송지점: {max(stops)}개")
    print(f"- 배송지점 범위: {max(stops) - min(stops)}개")
    print(f"- 평균 배송지점: {np.mean(stops):.1f}개")
    print(f"- 중앙값 배송지점: {np.median(stops):.0f}개")
    
    print("\n7. 효율성 지표")
    print(f"- km당 배송지점: {stats['total_points']/stats['total_distance']:.2f}개/km")
    print(f"- 시간당 배송지점: {stats['total_points']/(stats['total_time']/60):.2f}개/시간")
    print(f"- 차량당 평균 가동률: {stats['time_efficiency']:.1%}")
    
    # 지도 시각화
    print("\n=== 지도 시각화 생성 중 ===")
    m = visualize_routes(routes, [suwon_depot])
    return m

# 프로그램 실행
m = main(points, depots)
display(m)


=== 초기 데이터 정보 ===
전체 배송지점 수: 221개
물류센터 수: 8개

=== 수원 센터 기본 정보 ===
센터 위치: (37.263573, 127.028601)
배송권역 반경: 15km
배송지점 수: 221개

=== 최적화 결과 상세 분석 ===

1. 전체 운영 통계
- 총 배송지점: 221개
- 투입 차량 수: 7대
- 총 이동거리: 124.7km
- 총 소요시간: 1258분 (21.0시간)
- 시간 효율성: 42.8%

2. 차량당 평균 통계
- 평균 배송지점: 31.6개
- 평균 이동거리: 17.8km
- 평균 소요시간: 180분 (3.0시간)
- 소요시간 표준편차: 45.1분
- 소요시간 변동계수: 25.1%

3. 개별 차량 상세 정보

[차량 1번]
- 배송지점 수: 38개
- 이동거리: 20.8km
- 소요시간: 217분 (3.6시간)
- 상세 경로:
  출발: 수원센터 (37.263573, 127.028601)
  배송1: 경기도 수원시 권선구 권선로694번길 25 (권선동, 수원시청역 SK VIEW) (37.257779, 127.027665)
  배송2: 경기도 수원시 권선구 경수대로302번길 29 (권선동, 권선삼성아파트) (37.253907, 127.022824)
  배송3: 경기도 수원시 권선구 장다리로 20 (세류동) (37.252372, 127.018413)
  배송4: 경기도 수원시 권선구 세지로5번길 10-12 (세류동) (37.253514, 127.014469)
  배송5: 경기도 수원시 권선구 세지로5번길 10-12 (세류동) (37.253514, 127.014469)
  배송6: 경기도 수원시 권선구 정조로 410(세류동) (37.246710, 127.014985)
  배송7: 경기도 수원시 권선구 동수원로145번길 24(권선동, 수원아이파크시티2단지) (37.245120, 127.029458)
  배송8: 경기도 수원시 권선구 곡선로 10 (권선동, 수원아이파크시티5단지) (37.239160, 127.025